In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# My First Kaggle Notebook

# Customer Churn Prediction:

### Customer Churn means loss of cusomers. In this notebook, we try to predict the customer churn. The model gives 86% accuracy.

## Importing Libraries 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cv2

## Reading Dataset

In [ ]:
df= pd.read_csv('../input/bank-customers/Churn Modeling.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.drop(columns=['CustomerId','Surname','RowNumber'],inplace = True)

In [ ]:
df.dtypes


## Plot for relationship between Age and Exit Status

In [ ]:
Age_Exit = df[df.Exited == 1].Age
Age_NoExit = df[df.Exited == 0].Age
plt.hist([Age_Exit,Age_NoExit],color=['red','green'])

## Plot for relationship between Balance and Exit Status

In [ ]:
Balance_Exit = df[df.Exited == 1].Balance
Balance_NoExit = df[df.Exited == 0].Balance
plt.hist([Balance_Exit,Balance_NoExit],color=['red','green'])

## Plot for relationship between Gender and Exit Status

In [ ]:
Gender_Exit = df[df.Exited == 1]['Gender']
Gender_NoExit = df[df.Exited == 0]['Gender']
plt.hist([Gender_Exit,Gender_NoExit],color=['red','green'])

Females are exiting more than the males.

## Plot for relationship between Geography and Exit Status

In [ ]:
ES_Exit = df[df.Exited == 1].Geography
ES_NoExit = df[df.Exited == 0].Geography
plt.hist([ES_Exit,ES_NoExit],color=['red','green'])

In [ ]:
for column in df:
    print(f'{column}: {df[column].unique()}')

In [ ]:
df.shape

### One Hot Encoding for Geography and Gender Columns:

In [ ]:
df1 = pd.get_dummies(data = df,columns=['Geography','Gender'])

In [ ]:
df1.sample(4)

### Scaling Columns

In [ ]:
cols_to_scale = ['CreditScore','EstimatedSalary','Balance','Age']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df1[cols_to_scale] = scaler.fit_transform(df1[cols_to_scale])

### Spliting into X and Y and then using train_test_split

In [ ]:
X = df1.drop(columns=['Exited'],axis = 'columns')
y = df1['Exited']

In [ ]:
from sklearn.model_selection import train_test_split
X_Train,X_Test,Y_Train,Y_Test = train_test_split(X,y,test_size=0.2)

In [ ]:
X_Train.shape

### Model Formation

In [ ]:
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(32,input_shape = (13,),activation='relu'),
     keras.layers.Dense(16,activation='relu'),
     keras.layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

### Model Learning

In [ ]:
model.fit(X_Train,Y_Train,epochs = 50)

### Saving Model for next use

In [ ]:
model.save('save_model.h5')

In [ ]:
new_model = keras.models.load_model('save_model.h5')

In [ ]:
model.evaluate(X_Test,Y_Test)

### Predicting on Test Set and using Confusion Matrix and Classification Report for evaluting the performance of the model

In [ ]:
yp = model.predict(X_Test)

In [ ]:
y_pred = []
for element in yp:
    if element>0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(Y_Test,y_pred))

In [ ]:
import seaborn as sn
cm = tf.math.confusion_matrix(labels = Y_Test, predictions = y_pred)
sn.heatmap(cm,annot = True,fmt ='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

### The model gives around 86% accuracy.